<a href="https://colab.research.google.com/github/ratimayy/deep/blob/main/MLP2_ex1_4_WandB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- For data processing, model creation, and model training, this example is almost the same as MLP2_ex1-1. However, there are many new codes in this example regarding usages of the famous WandB (https://wandb.ai/).

- In this example you will learn the following new topics:
  - How to use the `dotenv` library to securely save your API key in a separated file
  - How to login to WandB (required a WandB account)
  - How to log hyperparameters of your project in each WandB's run
  - How to use `wandb.keras.WandbMetricsLogger()` as a Keras's callback to automatically log:
    - System (CPU/GPU/TPU) metrics
    - Train and validation metrics defined in `model.compile()`
    - Learning rate (both for a fixed value or a learning rate scheduler)
  - How to use `wandb.log()` to log custom values
  - How to use `wandb.keras.WandbModelCheckpoint()` as a Keras's callback to automatically save model's weights/checkpoints as WandB's artifacts
  - How to load the model/weights from WandB's artifact

# 1. Prepare the environment

## 1.1 Non-WandB preparation

In [ ]:
# List all NVIDIA GPUs as available in this computer (or Colab's session)
!nvidia-smi -L

GPU 0: NVIDIA L4 (UUID: GPU-d2756d80-2e16-9c2d-5dca-a999c6461430)


In [ ]:
import sys
print( f"Python {sys.version}\n" )

import os

import numpy as np
print( f"NumPy {np.__version__}\n" )

import pandas as pd
print( f"Pandas {pd.__version__}\n" )

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
print( f"Matplotlib {matplotlib.__version__}\n" )

Python 3.10.12 (main, Jul 29 2024, 16:56:48) [GCC 11.4.0]

NumPy 1.26.4

Pandas 2.1.4

Matplotlib 3.7.1



In [ ]:
# According to https://community.wandb.ai/t/about-wandbmodelcheckpoint-typeerror-modelcheckpoint-init-got-an-unexpected-keyword-argume/6677/6
# Using Keras 3 produces an error >> TypeError: ModelCheckpoint.__init__() got an unexpected keyword argument 'options'
# This is a known bug in WandB and it is suggested that we downgrade to Keras 2

# Install tensorflow with keras 2 according to https://keras.io/getting_started/
# Starting with TensorFlow 2.16, doing pip install tensorflow will install Keras 3.
!pip install "tensorflow<2.16"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Uninstalling ml-dtypes-0.4.0:
      Successfully uninstalled ml-dtypes-0.4.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installation

In [ ]:
import tensorflow as tf
print( f"TensorFlow {tf.__version__}" )
print( f"tf.keras.backend.image_data_format() = {tf.keras.backend.image_data_format()}" )

# Count the number of GPUs as detected by tensorflow
gpus = tf.config.list_physical_devices('GPU')
print( f"TensorFlow detected { len(gpus) } GPU(s):" )
for i, gpu in enumerate(gpus):
  print( f".... GPU No. {i}: Name = {gpu.name} , Type = {gpu.device_type}" )

TensorFlow 2.15.1
tf.keras.backend.image_data_format() = channels_last
TensorFlow detected 1 GPU(s):
.... GPU No. 0: Name = /physical_device:GPU:0 , Type = GPU


In [ ]:
# Set fixed seeding values for reproducability during experiments
# Skip this cell if random initialization (with varied results) is needed
#np.random.seed(1234)
#tf.random.set_seed(5678)

## 1.2 WandB preparation

In [ ]:
# Install WandB
!pip install wandb -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00


In [ ]:
# Import WandB
import wandb
print(f'WandB {wandb.__version__}')

WandB 0.17.7


In [ ]:
# (Optional) Install dotenv to help manage API keys stored in ".env" file
!pip install python-dotenv

**Start using WandB:**
- ~~Use as an anonymous and login by `wandb.login(anonymous='allow')`~~
  - ~~No WandB account required~~
  - ~~Limited features https://docs.wandb.ai/guides/app/features/anon~~
- Use with your personal WandB's API key (signing up required) and login by `wandb.login(key=YOUR_API_KEY_HERE)`
  - Retrieve your WandB API key from https://wandb.ai/authorize
  - (Optional) You may protect your API key by creating a text file named ".env" to store your API key like `WANDB_API_KEY = fdjkere12df4erdfdelre`. Then, use the `dotenv` library to load content in the ".env" file and get the API key.
- If you've already logged in to WandB in a web browser, most of the times, Google Colab will recognize your WandB account and automatically log you in without having to enter your WandB's API key manually.

In [ ]:
# Choose how you want to use WandB

menu = f"Do you have an environment variable named 'WANDB_API_KEY' stored in .env file?\nYour answer (y/n): "
while True:
  choice = input(menu).lower().strip()

  if choice == 'y':
    # Load 'WANDB_API_KEY' from the ".env" file
    from dotenv import load_dotenv
    load_dotenv()
    WANDB_API_KEY = os.getenv('WANDB_API_KEY')

    # Login with your WandB account
    wandb.login(key=WANDB_API_KEY)
    break

  elif choice == 'n':
    # Login as anonymous (no need for API key)
    wandb.login(anonymous='allow')
    break

  else:
    print('Invalid input. Try again.\n')

Do you have an environment variable named 'WANDB_API_KEY' stored in .env file?
Your answer (y/n): y


wandb: Currently logged in as: thitirat (gsas_nida). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# 2. Prepare the dataset

Load the Boston Housing dataset (http://lib.stat.cmu.edu/datasets/boston) from Keras

In [ ]:
# Load
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data()

# Inspect
vars = [ ('x_train',x_train), ('y_train',y_train), ('x_test',x_test), ('y_test',y_test) ]
for name,var in vars:
  print(f"{name}: type={type(var)} , shape={var.shape} , dtype={var.dtype}")

57026/57026 [==============================] - 0s 0us/step
x_train: type=<class 'numpy.ndarray'> , shape=(404, 13) , dtype=float64
y_train: type=<class 'numpy.ndarray'> , shape=(404,) , dtype=float64
x_test: type=<class 'numpy.ndarray'> , shape=(102, 13) , dtype=float64
y_test: type=<class 'numpy.ndarray'> , shape=(102,) , dtype=float64


(Optional) Preview the train and test sets with pandas' dataframe

In [ ]:
# A function to convert two numpy arrays to a single dataframe
def boston_dataframe(np_x, np_y):
  # Create a dataframe from two numpy arrays
  columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
  df_boston = pd.DataFrame(np_x, columns=columns)   # 13 feature columns
  df_boston['MEDV'] = pd.Series(np_y)   # 1 output column

  return df_boston

# Preview stats regarding both train and test datasets
print('===== PREVIEW: THE TRAIN SET =====')
display( boston_dataframe(x_train, y_train).describe().style.format(precision=2) )
print('\n===== PREVIEW: THE TEST SET =====')
display( boston_dataframe(x_test, y_test).describe().style.format(precision=2) )

===== PREVIEW: THE TRAIN SET =====


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,404.00,404.00,404.00,404.00,404.00,404.00,404.00,404.00,404.00,404.00,404.00,404.00,404.00,404.00
mean,3.75,11.48,11.10,0.06,0.56,6.27,69.01,3.74,9.44,405.90,18.48,354.78,12.74,22.40
std,9.24,23.77,6.81,0.24,0.12,0.71,27.94,2.03,8.70,166.37,2.20,94.11,7.25,9.21
min,0.01,0.00,0.46,0.00,0.39,3.56,2.90,1.13,1.00,188.00,12.60,0.32,1.73,5.00
25%,0.08,0.00,5.13,0.00,0.45,5.87,45.48,2.08,4.00,279.00,17.23,374.67,6.89,16.68
50%,0.27,0.00,9.69,0.00,0.54,6.20,78.50,3.14,5.00,330.00,19.10,391.25,11.39,20.75
75%,3.67,12.50,18.10,0.00,0.63,6.61,94.10,5.12,24.00,666.00,20.20,396.16,17.09,24.80
max,88.98,100.00,27.74,1.00,0.87,8.72,100.00,10.71,24.00,711.00,22.00,396.90,37.97,50.00



===== PREVIEW: THE TEST SET =====


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,102.00,102.00,102.00,102.00,102.00,102.00,102.00,102.00,102.00,102.00,102.00,102.00,102.00,102.00
mean,3.09,10.90,11.26,0.10,0.54,6.35,66.85,4.01,9.98,417.50,18.37,364.16,12.31,23.08
std,5.37,21.57,7.08,0.30,0.11,0.67,29.03,2.38,8.77,177.39,2.03,79.14,6.70,9.17
min,0.01,0.00,1.22,0.00,0.39,4.88,6.00,1.47,1.00,187.00,13.00,24.65,1.92,5.60
25%,0.08,0.00,5.46,0.00,0.45,5.97,42.45,2.12,4.00,279.25,17.40,377.69,7.30,18.65
50%,0.23,0.00,9.79,0.00,0.53,6.23,73.75,3.32,5.00,330.00,18.90,392.11,11.06,21.95
75%,3.78,16.25,18.10,0.00,0.61,6.63,92.97,5.28,24.00,666.00,20.20,396.78,15.91,27.08
max,25.05,90.00,27.74,1.00,0.87,8.78,100.00,12.13,24.00,711.00,21.20,396.90,31.99,50.00


Normalize each feature separately


In [ ]:
# Compute means and SDs from *the train set*
# 'axis' = Axis or axes along which the means are computed. The default is to compute the mean of the flattened array.
mean = x_train.mean(axis=0)   # https://numpy.org/doc/stable/reference/generated/numpy.mean.html
sd = x_train.std(axis=0)      # https://numpy.org/doc/stable/reference/generated/numpy.std.html
print(f"{mean.shape=} , {sd.shape=}\n")

# Normalize 13 features
x_train_norm = ( x_train - mean ) / sd
x_test_norm = ( x_test - mean ) / sd

# Inspect
print('===== PREVIEW: TRAIN SET AFTER NORM =====')
display( boston_dataframe(x_train_norm, y_train).describe().style.format(precision=2) )
print('\n===== PREVIEW: TEST SET AFTER NORM =====')
display( boston_dataframe(x_test_norm, y_test).describe().style.format(precision=2) )

mean.shape=(13,) , sd.shape=(13,)

===== PREVIEW: TRAIN SET AFTER NORM =====


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,404.00,404.00,404.00,404.00,404.00,404.00,404.00,404.00,404.00,404.00,404.00,404.00,404.00,404.00
mean,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,22.40
std,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,9.21
min,-0.41,-0.48,-1.56,-0.26,-1.47,-3.82,-2.37,-1.29,-0.97,-1.31,-2.67,-3.77,-1.52,5.00
25%,-0.40,-0.48,-0.88,-0.26,-0.89,-0.55,-0.84,-0.82,-0.63,-0.76,-0.57,0.21,-0.81,16.68
50%,-0.38,-0.48,-0.21,-0.26,-0.17,-0.10,0.34,-0.29,-0.51,-0.46,0.28,0.39,-0.19,20.75
75%,-0.01,0.04,1.03,-0.26,0.63,0.48,0.90,0.68,1.68,1.57,0.78,0.44,0.60,24.80
max,9.23,3.73,2.45,3.89,2.68,3.47,1.11,3.44,1.68,1.84,1.60,0.45,3.48,50.00



===== PREVIEW: TEST SET AFTER NORM =====


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,102.00,102.00,102.00,102.00,102.00,102.00,102.00,102.00,102.00,102.00,102.00,102.00,102.00,102.00
mean,-0.07,-0.02,0.02,0.15,-0.11,0.12,-0.08,0.13,0.06,0.07,-0.05,0.10,-0.06,23.08
std,0.58,0.91,1.04,1.24,0.94,0.95,1.04,1.17,1.01,1.07,0.92,0.84,0.92,9.17
min,-0.40,-0.48,-1.45,-0.26,-1.41,-1.96,-2.26,-1.12,-0.97,-1.32,-2.49,-3.51,-1.49,5.60
25%,-0.40,-0.48,-0.83,-0.26,-0.95,-0.42,-0.95,-0.80,-0.63,-0.76,-0.49,0.24,-0.75,18.65
50%,-0.38,-0.48,-0.19,-0.26,-0.22,-0.05,0.17,-0.20,-0.51,-0.46,0.19,0.40,-0.23,21.95
75%,0.00,0.20,1.03,-0.26,0.44,0.52,0.86,0.76,1.68,1.57,0.78,0.45,0.44,27.08
max,2.31,3.31,2.45,3.89,2.68,3.54,1.11,4.14,1.68,1.84,1.24,0.45,2.66,50.00


For data to be processed by the model, convert them to float32 dtype

In [ ]:
# A conversion function
convert = lambda a: a.astype(np.float32)

# Convert and inspect
vars = [ ('x_train_norm',x_train_norm), ('y_train',y_train), ('x_test_norm',x_test_norm), ('y_test',y_test) ]
for name,var in vars:
  var = convert(var)
  print(f"{name}: type={type(var)} , shape={var.shape} , dtype={var.dtype}")

x_train_norm: type=<class 'numpy.ndarray'> , shape=(404, 13) , dtype=float32
y_train: type=<class 'numpy.ndarray'> , shape=(404,) , dtype=float32
x_test_norm: type=<class 'numpy.ndarray'> , shape=(102, 13) , dtype=float32
y_test: type=<class 'numpy.ndarray'> , shape=(102,) , dtype=float32


# 3. Create the network architecture

In [ ]:
def create_model(config):
  '''
    config (wandb.config): related hyperparameters to be logged by WandB
  '''
  # Model creation
  tf.keras.backend.clear_session()
  model = tf.keras.models.Sequential(name=config.model_name)

  # Input layer
  model.add( tf.keras.Input(shape=(x_train_norm.shape[-1],), name='input') )

  # Hidden layers
  model.add( tf.keras.layers.Dense(8, activation='relu', name='hidden1') )   # use default weight initialization, don't use any regularization
  model.add( tf.keras.layers.BatchNormalization(axis=-1, name='bn1') )
  model.add( tf.keras.layers.Dense(16, activation='relu', name='hidden2') )  # use default weight initialization, don't use any regularization
  model.add( tf.keras.layers.BatchNormalization(axis=-1, name='bn2') )
  model.add( tf.keras.layers.Dense(8, activation='relu', name='hidden3') )   # use default weight initialization, don't use any regularization
  model.add( tf.keras.layers.Dropout(0.3) )          # drop rate = 30%

  # Output layer: For a regression problem, use a linear activation function (a.k.a., use no activation function)
  model.add( tf.keras.layers.Dense(1, name='output') )

  # Compile
  model.compile( optimizer=config.optimizer, loss=config.loss, metrics=[config.metric] )

  return model

# 4. Train and evaluate models with WandB

In [ ]:
menu = '\n1. Save weights only\n2. Save the full model\nYour choice: '

while True:
  choice = input(menu).strip()
  if choice in ['1', '2']:
    save_weights_only = True if choice=='1' else False
    break


1. Save weights only
2. Save the full model
Your choice: 2


In [ ]:
def load_newest_model_from_disk(config, dir='./'):
  '''
    config (wandb.config): without this there will be error on 'create_model()'
    dir (str): directory of the file to load
  '''
  if save_weights_only:
    # Get the name of the newest .weights.h5 file
    files = [os.path.join(dir,x) for x in os.listdir(dir) if x.endswith(".weights.h5")]
    newest = max(files , key=os.path.getctime)

    # Load weights of the newest model
    newest_model = create_model(config)   # Load the network architecture
    newest_model.load_weights(newest)     # Load weights
    print(f"Successfully loaded the newest model from '{newest}'")
  else:
    # Get the name of the newest .keras file
    files = [os.path.join(dir,x) for x in os.listdir(dir) if x.endswith(".keras")]
    newest = max(files , key=os.path.getctime)

    # Load the newest model
    newest_model = tf.keras.models.load_model(newest)
    print(f"Successfully loaded the newest model from '{newest}'")

  return newest_model

In [ ]:
def experiment_with_wandb(project_name, config_dict):
  '''
    project_name (str): Name of WandB project
    config_dict (dict): A python dict of WandB configurations/hyperparameters
  '''
  # Start a run: do it manually by calling wandb.init() or using the with block
  #run = wandb.init(project=project_name, config=config_dict)
  with wandb.init(project=project_name, config=config_dict) as run:
    # Build a model
    model = create_model(run.config)
    #model.summary()

    # Prepare the path to save model's checkpoints
    if save_weights_only:
      checkpoint_filepath = "bestmodel_epoch{epoch:03d}_valloss{val_loss:.2f}.weights.h5"
      #checkpoint_filepath = "bestmodel.weights.h5"
    else:
      checkpoint_filepath = "bestmodel_epoch{epoch:03d}_valloss{val_loss:.2f}.keras"
      #checkpoint_filepath = "bestmodel.keras"

    # Add WandbMetricsLogger to log metrics
    # Add WandbModelCheckpoint to log model checkpoints/weights
    # |-- Note that, the model will be logged and saved to both WandB and your local computer
    wandb_callbacks = [ wandb.keras.WandbMetricsLogger(),
                        wandb.keras.WandbModelCheckpoint(
                                        filepath=checkpoint_filepath,
                                        save_weights_only=save_weights_only,
                                        monitor='val_loss',
                                        mode='min',
                                        save_best_only=True
                                  ),
                      ]

    # Train the model with two WandB callbacks
    model.fit ( x_train_norm, y_train,
                validation_split=run.config.val_split,
                batch_size=run.config.batch_size, epochs=run.config.epoch, verbose=0,
                callbacks=wandb_callbacks
              )

    # Evaluate the trained model (from the last epoch) on the test set
    # Log the results to WandB
    results = model.evaluate(x_test_norm, y_test, batch_size=50, verbose=0)
    run.log( { f'test_{model.metrics_names[0]}': results[0],
               f'test_{model.metrics_names[1]}': results[1]  }  )

    # Evaluate the trained model (from the best epoch) on the test set
    # Log the results to WandB
    best_model = load_newest_model_from_disk(run.config)
    results = best_model.evaluate(x_test_norm, y_test, batch_size=50, verbose=0)
    run.log( { f'test_best_{model.metrics_names[0]}': results[0],
               f'test_best_{model.metrics_names[1]}': results[1]  }  )

    # Manually mark the run as finished
    #run.finish()   # This line is not necessary when using the with block

Try running several experiments with different configurations
- To log multiple metrics on the same chart (e.g., train loss and val loss), follow https://community.wandb.ai/t/log-multiple-variables-at-the-same-plot/2474. However, this cannot be done by an anonymous user.

In [ ]:
# Use a Python dict to store whatever hyperparameters you want to log in this project
default_configs = dict( model_name='MLP2',
                        dataset='Boston',
                        val_split=0.2,
                        optimizer='adam',
                        loss='mean_squared_error',
                        metric='mean_absolute_error',
                        epoch=5,
                        batch_size=50
                  )

In [ ]:
# First experiment
experiment_with_wandb('DLclass_MLP2', default_configs)

Successfully loaded the newest model from './bestmodel_epoch005_valloss638.85.keras'


epoch/epoch,▁▃▅▆█
epoch/learning_rate,▁▁▁▁▁
epoch/loss,█▆▄▂▁
epoch/mean_absolute_error,█▆▄▂▁
epoch/val_loss,█▇▅▃▁
epoch/val_mean_absolute_error,█▇▅▃▁
test_best_loss,▁
test_best_mean_absolute_error,▁
test_loss,▁
test_mean_absolute_error,▁
epoch/epoch,4


In [ ]:
# Second experiment
default_configs['batch_size'] = 100
default_configs['epoch'] = 100
experiment_with_wandb('DLclass_MLP2', default_configs)

Successfully loaded the newest model from './bestmodel_epoch100_valloss42.23.keras'


epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,████▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁
epoch/mean_absolute_error,████▇▇▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▃▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁
epoch/val_loss,███████▇▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
epoch/val_mean_absolute_error,█████████▇▇▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁
test_best_loss,▁
test_best_mean_absolute_error,▁
test_loss,▁
test_mean_absolute_error,▁
epoch/epoch,99


In [ ]:
# Third experiment
default_configs['batch_size'] = 100
default_configs['epoch'] = 20
experiment_with_wandb('DLclass_MLP2', default_configs)

Successfully loaded the newest model from './bestmodel_epoch020_valloss589.66.keras'


epoch/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,███▇▇▆▆▆▅▅▄▄▄▃▃▂▂▂▁▁
epoch/mean_absolute_error,███▇▇▆▆▆▅▅▅▄▄▃▃▂▂▂▁▁
epoch/val_loss,██▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▂▂▁
epoch/val_mean_absolute_error,███▇▇▇▆▆▆▆▅▅▄▄▄▃▃▂▂▁
test_best_loss,▁
test_best_mean_absolute_error,▁
test_loss,▁
test_mean_absolute_error,▁
epoch/epoch,19


# 5. Load and use WandB's artifacts

1. Go to the `Artifacts` tab in WandB
2. In the target run, choose the file you want to retrieve. Then,
  - Under the `Version` tab, copy the url specified in the `Full Name` field and paste it in the cell below
  - Find example codes under the `Usage` tab

In [ ]:
# Below is just a dummy url, DO NOT USE IT
# Instead, replace the below url with YOUR URL from WandB
FILE_URL = 'gsas_nida/DLclass_MLP2/run_zp94vvo4_model:v19'

In [ ]:
# Download the file from this WandB artifact to your local computer
run = wandb.init(config=default_configs)
artifact = run.use_artifact(FILE_URL, type='model')
artifact_dir = artifact.download()
run.finish()

print(f'{artifact_dir=}')

wandb:   1 of 1 files downloaded.  


artifact_dir='/content/artifacts/run_zp94vvo4_model:v19'


In [ ]:
# Load the model and use it
artifact_model = load_newest_model_from_disk(run.config, dir=artifact_dir)
results = artifact_model.evaluate(x_test_norm, y_test, batch_size=50)
print( f"{artifact_model.metrics_names} = {results}" )

Successfully loaded the newest model from '/content/artifacts/run_zp94vvo4_model:v19/bestmodel_epoch020_valloss589.66.keras'
3/3 [==============================] - 0s 4ms/step - loss: 565.8778 - mean_absolute_error: 22.0659
['loss', 'mean_absolute_error'] = [565.8778076171875, 22.065919876098633]
